In [1]:
import pandas as pd
import geopandas

In [2]:
# world_map = geopandas.read_file('filtered_countries.geojson')
# world_map.loc[world_map['name'].str.contains('France')].iloc[0,2]

In [3]:
countries = {
    'BR': {'country': 'BR', 'country_name': 'Brazil', 'latitude': -15.749997, 'longitude': -51.92528}, 
    'CA': {'country': 'CA', 'country_name': 'Canada', 'latitude': 56.130366, 'longitude': -106.346771},
    'DE': {'country': 'DE', 'country_name': 'Germany', 'latitude': 51.165691, 'longitude': 10.451526},
    'FR': {'country': 'FR', 'country_name': 'France', 'latitude': 46.227638, 'longitude': 2.213749},
    'GB': {'country': 'GB', 'country_name': 'UK', 'latitude': 55.378051, 'longitude': -3.435973},
    'IN': {'country': 'IN', 'country_name': 'India', 'latitude': 20.593684, 'longitude': 78.96288},
    'JP': {'country': 'JP', 'country_name': 'Japan', 'latitude': 36.204824, 'longitude': 138.252924},
    'KR': {'country': 'KR', 'country_name': 'S Korea', 'latitude': 35.907757, 'longitude': 127.766922},
    'MX': {'country': 'MX', 'country_name': 'Mexico', 'latitude': 23.634501, 'longitude': -102.552784},
    'RU': {'country': 'RU', 'country_name': 'Russia', 'latitude': 61.52401, 'longitude': 105.318756},
    'US': {'country': 'US', 'country_name': 'USA', 'latitude': 37.09024, 'longitude': -95.712891}}

countries_df = pd.DataFrame(countries.values())

# countries_df.to_csv('countries.csv', index = False)

# countries_df

df = pd.read_csv('df_youtube_raw.csv')

df = df.drop(columns = [df.columns[0], 'publish_time', 'video_id', 'thumbnail_link', 
                         'ratings_disabled', 'comments_disabled', 'description'])

df = df.merge(countries_df, on='country')
df = df.drop(columns = ['country'])

df['views'] = pd.to_numeric(df['views'], downcast='integer')
df['likes'] = pd.to_numeric(df['likes'], downcast='integer')
df['dislikes'] = pd.to_numeric(df['dislikes'], downcast='integer')
df['comment_count'] = pd.to_numeric(df['comment_count'], downcast='integer')
# df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
df['trending_date'] = pd.to_datetime(df['trending_date'], format='%Y-%m-%d')

# df = df.loc[df['trending_date'] > '2020-01-01']
df = df.dropna()

def transformTags(x):
#     return "|".join(x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|'))
    return x.strip().replace('"', '').replace('[None]', '').replace('[none]', '').lower().split('|')

df['tags'] = df['tags'].apply(transformTags)

In [4]:
bubble_map_data = df.groupby(['country_name', 'latitude', 'longitude']).agg({'title': 'count'}).rename(columns = {'title': 'value'})

bubble_map_data.to_csv('bubble_map_data.csv', index = True)

pd.read_csv('bubble_map_data.csv')

,country_name,latitude,longitude,value
0,Brazil,-15.749997,-51.925280,46887
1,Canada,56.130366,-106.346771,87612
2,France,46.227638,2.213749,87424
3,Germany,51.165691,10.451526,87357
4,India,20.593684,78.962880,77034
5,Japan,36.204824,138.252924,61642
6,Mexico,23.634501,-102.552784,86824
7,Russia,61.524010,105.318756,84877
8,S Korea,35.907757,127.766922,75687
9,UK,55.378051,-3.435973,85501


In [5]:
doughnut = df.groupby(['country_name', 'category_title']).agg({'title': 'count'}).rename(columns = {'title': 'count'}).reset_index()

doughnut = doughnut.merge(bubble_map_data, on = 'country_name').rename(columns = {'value' : 'total'})

doughnut['value'] = round(doughnut['count'] / doughnut['total'] * 100, 1)

doughnut.index = doughnut['country_name']

doughnut = doughnut.drop(columns = ['country_name', 'count', 'total'])

doughnut_temp = df.groupby(['category_title']).agg({'title': 'count'}).rename(columns = {'title': 'value'}).reset_index()

doughnut_temp['value'] = round(doughnut_temp['value'] / doughnut_temp.sum()['value'] * 100, 1)

doughnut_temp.index = ['Overall'] * doughnut_temp.shape[0]

doughnut_temp.index.name = 'country_name'

doughnut = doughnut.append(doughnut_temp)

main_list = []

for i in doughnut.index.unique():    
    as_list = doughnut.loc[i]['category_title'].tolist()
    
    for j in range(len(as_list)):        
        if doughnut.loc[i][doughnut.loc[i].category_title.str.contains(as_list[j])]['value'][0] < 3.5:
            as_list[j] = 'Others'

    main_list += as_list


doughnut['category_title'] = main_list

doughnut = doughnut.groupby(['country_name', 'category_title']).sum().round(1).sort_values(['country_name', 'value'], ascending = [True, False])

doughnut.to_csv('doughnut.csv', index = True)

pd.read_csv('doughnut.csv')

doughnut.head(20)

value
country_name category_title             
Brazil       Music                  25.8
             Entertainment          25.5
             Sports                 15.3
             People & Blogs         12.0
             Others                  9.5
             Gaming                  7.1
             Comedy                  4.8
Canada       Entertainment          26.7
             Music                  12.7
             Sports                  9.9
             People & Blogs          9.3
             Comedy                  8.0
             Gaming                  7.8
             News & Politics         6.8
             Others                  5.8
             Howto & Style           4.8
             Film & Animation        4.1
             Science & Technology    3.9
France       Entertainment          23.9
             Music                  17.4

In [6]:
def flatList(x):
    return [item for sublist in x for item in sublist if len(item) > 1]

wordcloud = pd.DataFrame(df.groupby('country_name')['tags'].apply(list))
wordcloud = pd.DataFrame(wordcloud['tags'].apply(flatList))
wordcloud.loc['Overall'] = [[item for list in wordcloud['tags'].to_list() for item in list]]

def topNWords(x, n = 80):
    return pd.DataFrame(x[0]).value_counts().sort_values(ascending = False).head(n)

wordcloud = pd.DataFrame(wordcloud.groupby('country_name')['tags'].apply(topNWords)).reset_index().rename(columns = {0: 'words', 'tags': 'count'})
wordcloud.index = wordcloud.country_name
wordcloud[['words', 'count']].to_csv('wordcloud.csv', index = True)
pd.read_csv('wordcloud.csv')

# wordcloud = wordcloud[['words', 'count']].groupby('country_name')['words'].apply(list).apply(lambda x: "|".join(x))

,country_name,words,count
0,Brazil,futebol,2442
1,Brazil,vlog,1904
2,Brazil,flamengo,1597
3,Brazil,humor,1536
4,Brazil,ao vivo,1485
...,...,...,...
955,USA,film,859
956,USA,animated,859
957,USA,snl,857
958,USA,love,850


['comments', 'likes', 'dislikes', 'views']

In [53]:
bubble_plot_data = df.groupby(['country_name']).agg({'comment_count': 'sum', 'likes': 'sum', 'dislikes': 'sum', 'views': 'sum'}).rename(columns = {'comment_count': 'comments'})

bubble_plot_data.to_csv('bubble_plot_data.csv', index = True)

pd.read_csv('bubble_plot_data.csv')

,country_name,comments,likes,dislikes,views
0,Brazil,493155157,5.681667e+09,103664067,7.285022e+10
1,Canada,906322982,8.448183e+09,233456795,1.635380e+11
2,France,445232589,4.488384e+09,105460222,6.742831e+10
3,Germany,569776607,4.946625e+09,159922045,8.830088e+10
4,India,738714001,6.713708e+09,412267150,1.510288e+11
5,Japan,364043084,2.730572e+09,68199385,5.591028e+10
6,Mexico,736729044,7.620644e+09,172462532,1.081063e+11
7,Russia,340027141,2.894440e+09,127910580,3.965003e+10
8,S Korea,518018409,3.743043e+09,83661656,7.736269e+10
9,UK,1060897736,1.056836e+10,407795649,3.191453e+11


In [107]:
stacked_barplot = df.groupby(['country_name', 'category_title']).agg({'comment_count': 'sum', 'likes': 'sum', 'dislikes': 'sum', 'views': 'sum'}).rename(columns = {'comment_count': 'comments'})

overall_engagement = stacked_barplot.reset_index()

overall_engagement['country_name'] = overall_engagement['country_name'].apply(lambda x: "Overall")

overall_engagement = overall_engagement.groupby(['country_name', 'category_title']).agg({'comments': 'sum', 'likes': 'sum', 'dislikes': 'sum', 'views': 'sum'}).rename(columns = {'comment_count': 'comments'})

stacked_barplot = stacked_barplot.append(overall_engagement)

stacked_barplot['engagement_rate'] = round(((stacked_barplot['comments'] / stacked_barplot['views']) + \
                                    (stacked_barplot['likes'] / stacked_barplot['views']) + \
                                    (stacked_barplot['dislikes'] / stacked_barplot['views'])) * 100, 2)

stacked_barplot['total_engagement'] = stacked_barplot['comments'] + stacked_barplot['likes'] + stacked_barplot['dislikes']

for i in list(stacked_barplot.columns[:3]):
    stacked_barplot[i] = round(stacked_barplot[i] / stacked_barplot['total_engagement'] * 100, 2)

stacked_barplot = stacked_barplot.dropna()

stacked_barplot.iloc[:,:-1].to_csv('stacked_barplot.csv', index = True)
pd.read_csv('stacked_barplot.csv')

,country_name,category_title,comments,likes,dislikes,views,engagement_rate
0,Brazil,Autos & Vehicles,4.99,94.06,0.95,1.321137e+08,11.77
1,Brazil,Comedy,3.42,95.02,1.56,2.051216e+09,10.54
2,Brazil,Education,4.81,93.59,1.60,1.791079e+08,10.08
3,Brazil,Entertainment,5.52,93.15,1.33,1.211587e+10,10.77
4,Brazil,Film & Animation,6.31,91.93,1.76,8.177259e+08,5.69
...,...,...,...,...,...,...,...
184,Overall,Science & Technology,8.17,88.72,3.11,4.374808e+10,4.07
185,Overall,Shows,11.02,77.83,11.15,4.026953e+08,1.38
186,Overall,Sports,8.53,88.04,3.42,7.988343e+10,2.79
187,Overall,Trailers,0.96,94.74,4.31,5.504300e+04,0.38


comments  likes  dislikes         views  \
country_name category_title                                                  
Brazil       Autos & Vehicles          4.99  94.06      0.95  1.321137e+08   
             Comedy                    3.42  95.02      1.56  2.051216e+09   
             Education                 4.81  93.59      1.60  1.791079e+08   
             Entertainment             5.52  93.15      1.33  1.211587e+10   
             Film & Animation          6.31  91.93      1.76  8.177259e+08   
...                                     ...    ...       ...           ...   
Overall      Science & Technology      8.17  88.72      3.11  4.374808e+10   
             Shows                    11.02  77.83     11.15  4.026953e+08   
             Sports                    8.53  88.04      3.42  7.988343e+10   
             Trailers                  0.96  94.74      4.31  5.504300e+04   
             Travel & Events           9.58  87.77      2.65  2.097969e+09   

                                   engagement_rate  total_engagement  
country_name category_title                                           
Brazil       Autos & Vehicles                11.77      1.554351e+07  
             Comedy                          10.54      2.162435e+08  
             Education                       10.08      1.805629e+07  
             Entertainment                   10.77      1.305121e+09  
             Film & Animation                 5.69      4.656387e+07  
...                                            ...               ...  
Overall      Science & Technology             4.07      1.778978e+09  
             Shows                            1.38      5.557113e+06  
             Sports                           2.79      2.227994e+09  
             Trailers                         0.38      2.090000e+02  
             Travel & Events                  4.56      9.568253e+07  

[189 rows x 6 columns]

In [9]:
# def expand(x):
#     return x

# arr = list(df.loc[df['country_name'] == 'Germany']['tags'].apply(expand))
# all_tags = [item.lower() for a in arr for item in a]

# df_alltags = pd.DataFrame(all_tags).rename(columns = {0: 'tags'})
# all_tags_count = pd.DataFrame(df_alltags.value_counts())

In [10]:
# all_tags_count.head(40)

# all_tags_count.filter(like = 'none', axis = 0)

# df.filter(like = 'index to keep', axis=0)